In [21]:
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
from bertopic import BERTopic
import pandas as pd
import numpy as np
from transformers import pipeline

In [22]:
embedding_model = SentenceTransformer('NbAiLab/nb-sbert-base')
topic_model = BERTopic(embedding_model=embedding_model, verbose=True)
keyword_model = KeyBERT(model=topic_model)
ner_model = pipeline(task='ner', 
               model='saattrupdan/nbailab-base-ner-scandi', 
               aggregation_strategy='first')

In [23]:
def ner_lambda(x: str):
    try:
        pred = ner_model(x)
        result = {}
        if pred is not None:
            for d in pred:
                result[d['word']] = d['entity_group']
        return result
    except Exception as e:
        print(f"Exception: {e}\nCaused by: {x}")
        return {}

In [24]:
df = pd.read_csv('HLE_m_annonser.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.dropna(inplace=True)
df.head()

,Link,Headline,Category,Subtitle,Keywords
0,https://www.dagbladet.no/nyheter/kaotisk-nytta...,kaotisk nyttarsnatt ute av kontroll,nyheter,Det har vært en hektisk nyttårsnatt for politi...,nyheter
1,https://www.dagbladet.no/tema/fra-singel-til-d...,fra singel til damemagnet,tema,Ekspertenes beste råd til hvordan du blir attr...,"kjæreste,dating,singel"
2,https://www.dagbladet.no/tema/strom-nytt-fra-j...,strom nytt fra januar,tema,"Nye boligregler, bedre jobbpensjon og dyrere f...","økonomi,pensjon,skatt"
3,https://www.dagbladet.no/tema/anders-kvitt-mag...,anders kvitt magefettet i superfart 1,tema,"- Hvis folk bare visste hvor enkelt det er, ha...","magefett,vektnedgang,kosthold,styrketrening,"
4,https://www.dagbladet.no/nyheter/taus-om-overg...,taus om overgrepsdom,nyheter,Trump ønsket «henne alt godt» i fjor sommer. N...,"donald trump,ghislaine maxwell,utenriks,nyhete..."


In [25]:
n_adverts = df[df['Category'] == 'annonse'].shape[0]
print(f"Percentage of adverts in dataset: {n_adverts/df.shape[0]*100:.2f}%")

Percentage of adverts in dataset: 5.00%


In [26]:
df['subtitle_entities'] = df['Subtitle'].apply(lambda x: ner_lambda(x))

Exception: At least one input is required.
Caused by: nan
Exception: At least one input is required.
Caused by: nan


In [27]:
df['keywords_entities'] = df['Keywords'].apply(lambda x: ner_lambda(x))

Exception: 'NoneType' object is not iterable
Caused by:  
Exception: 'NoneType' object is not iterable
Caused by:  


In [34]:
# Create a list with all docs from df.Subtitle
docs = df.Subtitle.tolist()
topic_model.fit(docs)

Batches: 100%|██████████| 532/532 [19:52<00:00,  2.24s/it]
2022-12-09 16:33:02,809 - BERTopic - Transformed documents to Embeddings
2022-12-09 16:33:13,652 - BERTopic - Reduced dimensionality
2022-12-09 16:33:15,055 - BERTopic - Clustered reduced embeddings


In [32]:
# Check if docs-list has a float value
for i, doc in enumerate(docs):
    if isinstance(doc, float):
        print(f"Found float at index {i}. Value: {doc}")


Found float at index 16805. Value: nan
Found float at index 16894. Value: nan


In [36]:
topic_model.save('topicModel')

In [37]:
topic_model.visualize_topics()

In [38]:
subtitle_topics = topic_model.transform(docs)

Batches: 100%|██████████| 532/532 [20:06<00:00,  2.27s/it]
2022-12-09 16:55:25,246 - BERTopic - Reduced dimensionality
2022-12-09 16:55:25,830 - BERTopic - Predicted clusters


In [39]:
df['BERT_topic_num'] = -1
df['BERT_topic_prob'] = 0
for i in range(len(subtitle_topics[0])):
    group = subtitle_topics[0][i]
    prob = subtitle_topics[1][i]
    df['BERT_topic_num'][i] = group
    df['BERT_topic_prob'][i] = prob

In [40]:
# Read 'stopwords-no.txt' and create a list of stopwords
with open('stopwords-no.txt', 'r') as f:
    stopwords = f.read().splitlines()


In [41]:
keywords = keyword_model.extract_keywords(docs, keyphrase_ngram_range=(1, 1), stop_words=stopwords, top_n=5)

In [42]:
# Write the keywords to a file called subtitles_keywords.txt
with open('subtitles_keywords.txt', 'w', encoding='utf8') as f:
    f.write(str(keywords))

In [43]:
df['BERT_keywords'] = ''
for i in range(len(keywords)):
    current_keywords = keywords[i]
    df['BERT_keywords'][i] = [tup[0] for tup in current_keywords]

In [44]:
bert_kw = df.BERT_keywords.tolist()

In [56]:
df['BERT_kw_topics'] = ''
k = 0
for i in range(k, len(bert_kw)):
    try:
        kw = bert_kw[i]
        if len(kw) == 0:
            continue
        if len(kw) == 1:
            kw = kw[0]
        topic_nums, topic_probs = topic_model.find_topics(kw, top_n=5)
        df['BERT_kw_topics'][i] = topic_nums
    except Exception as e:
        print(f"Cause: {kw}")
        raise e

In [57]:
bert_df_kw = df.BERT_keywords
bert_kw_topics = df.BERT_kw_topics

In [58]:
df['BERT_keywords'] = bert_df_kw
df['BERT_kw_topics'] = bert_kw_topics

In [62]:
df.head()

,Link,Headline,Category,Subtitle,Keywords,subtitle_entities,keywords_entities,BERT_topic_num,BERT_topic_prob,BERT_keywords,BERT_kw_topics
0,https://www.dagbladet.no/nyheter/kaotisk-nytta...,kaotisk nyttarsnatt ute av kontroll,nyheter,Det har vært en hektisk nyttårsnatt for politi...,nyheter,{},{},-1,0.000000,"[nyttårsnatt, politiet, hektisk, landet, flere]","[22, 120, 89, 109, 159]"
1,https://www.dagbladet.no/tema/fra-singel-til-d...,fra singel til damemagnet,tema,Ekspertenes beste råd til hvordan du blir attr...,"kjæreste,dating,singel",{},{},12,0.300247,"[datingmarkedet, attraktiv, råd, ekspertenes, ...","[140, 10, 59, 68, 85]"
2,https://www.dagbladet.no/tema/strom-nytt-fra-j...,strom nytt fra januar,tema,"Nye boligregler, bedre jobbpensjon og dyrere f...","økonomi,pensjon,skatt",{},{},-1,0.000000,"[boligregler, endringer, dyrere, bedre, nye]","[154, 96, 136, 166, 104]"
3,https://www.dagbladet.no/tema/anders-kvitt-mag...,anders kvitt magefettet i superfart 1,tema,"- Hvis folk bare visste hvor enkelt det er, ha...","magefett,vektnedgang,kosthold,styrketrening,",{'Anders Muren': 'PER'},{},98,0.939760,"[magefettet, overflødige, enkelt, raskt, visste]","[98, 79, 18, 80, 137]"
4,https://www.dagbladet.no/nyheter/taus-om-overg...,taus om overgrepsdom,nyheter,Trump ønsket «henne alt godt» i fjor sommer. N...,"donald trump,ghislaine maxwell,utenriks,nyhete...","{'Trump': 'PER', 'Ghislaine Maxwell': 'PER', '...","{'donald trump': 'PER', 'usa': 'LOC', 'jeffrey...",93,0.785539,"[maxwell, trump, presidenten, unge, dømt]","[8, 93, 7, 32, 54]"


In [59]:
df.to_csv('Preprocessed_Data.csv', index=False)

In [60]:
df.head()

,Link,Headline,Category,Subtitle,Keywords,subtitle_entities,keywords_entities,BERT_topic_num,BERT_topic_prob,BERT_keywords,BERT_kw_topics
0,https://www.dagbladet.no/nyheter/kaotisk-nytta...,kaotisk nyttarsnatt ute av kontroll,nyheter,Det har vært en hektisk nyttårsnatt for politi...,nyheter,{},{},-1,0.000000,"[nyttårsnatt, politiet, hektisk, landet, flere]","[22, 120, 89, 109, 159]"
1,https://www.dagbladet.no/tema/fra-singel-til-d...,fra singel til damemagnet,tema,Ekspertenes beste råd til hvordan du blir attr...,"kjæreste,dating,singel",{},{},12,0.300247,"[datingmarkedet, attraktiv, råd, ekspertenes, ...","[140, 10, 59, 68, 85]"
2,https://www.dagbladet.no/tema/strom-nytt-fra-j...,strom nytt fra januar,tema,"Nye boligregler, bedre jobbpensjon og dyrere f...","økonomi,pensjon,skatt",{},{},-1,0.000000,"[boligregler, endringer, dyrere, bedre, nye]","[154, 96, 136, 166, 104]"
3,https://www.dagbladet.no/tema/anders-kvitt-mag...,anders kvitt magefettet i superfart 1,tema,"- Hvis folk bare visste hvor enkelt det er, ha...","magefett,vektnedgang,kosthold,styrketrening,",{'Anders Muren': 'PER'},{},98,0.939760,"[magefettet, overflødige, enkelt, raskt, visste]","[98, 79, 18, 80, 137]"
4,https://www.dagbladet.no/nyheter/taus-om-overg...,taus om overgrepsdom,nyheter,Trump ønsket «henne alt godt» i fjor sommer. N...,"donald trump,ghislaine maxwell,utenriks,nyhete...","{'Trump': 'PER', 'Ghislaine Maxwell': 'PER', '...","{'donald trump': 'PER', 'usa': 'LOC', 'jeffrey...",93,0.785539,"[maxwell, trump, presidenten, unge, dømt]","[8, 93, 7, 32, 54]"
